# Visualizing Geospatial Meterological Data in Python : Temperature, Wind and Snow Mask

This notenook shows how to use several open source tools and techniques for visualizing meterological data from a shapefile on differents types of maps.

For this, we will use :
- [BaseMap](https://matplotlib.org/basemap/stable/) to plot simple variable like temperature
- [Cartopy](https://scitools.org.uk/cartopy/docs/latest/index.html) to plot complexe variable like wind
- [Cartopy](https://scitools.org.uk/cartopy/docs/latest/index.html) to plot a snow mask on a S2 image
- [Folium](https://python-visualization.github.io/folium/latest/) to plot interactive map with a snow mask and S2 tile
- [Seaborn](https://seaborn.pydata.org/) to plot statistical data 


## Database initialisation and simplification

The meterological data grib come from ECMWF "ERA5-Land hourly data from 1950 to present" database on 12 december 2023 at 12h with :
- 10 metre U wind component (m/s)
- 10 metre V wind component (m/s)
- 2 metre temperature (K)
- Snow albedo (0-1) 
- Snow cover (%)
- Surface pressure (Pa)
- Total precipitation (m)

The link to the dataset: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land

The link of the S2 catalogue used at the end: https://catalogue.theia-land.fr/

In [ ]:
import numpy as np                       # linear algebra
import matplotlib.pyplot as plt          # plotting library
import pandas as pd                      # data processing
import geopandas as gpd                  # data processing

plt.ion();

In [ ]:
import pygrib                            # era5 grib processing       

# Import data
file = 'data/all_var_1212.grib'
grbs = pygrib.open(file)

# Get Lat/Lon
lats, lons = grbs.select()[0].latlons()
lats, lons = np.array(lats), np.array(lons)

# Get Variables
data_uwind = grbs.select()[0].values
data_vwind = grbs.select()[1].values
data_t2m = grbs.select()[2].values
data_snowa = grbs.select()[3].values
data_snowc = grbs.select()[4].values
data_press = grbs.select()[5].values
data_prec = grbs.select()[6].values

# Translate all no-data values to np.nan
data_t2m[data_t2m > 1000] = np.nan
data_t2m = data_t2m - 273.15 # substrat zero kelvin
data_uwind[data_uwind > 1000] = np.nan
data_snowa[data_snowa > 1000] = np.nan
data_snowc[data_snowc > 1000] = np.nan
data_vwind[data_vwind > 1000] = np.nan
data_press[data_press > 106000] = np.nan

## Plot examples

### Plot a simple variable like temperature

Here is a simple way to plot a simple variable like 2m temperature on a map with Basemap

In [ ]:
from mpl_toolkits.basemap import Basemap

m = Basemap(projection='cyl', llcrnrlon=-5, llcrnrlat=40, urcrnrlon=5, urcrnrlat=47, resolution='i')
m.drawcoastlines(1)
m.drawcountries()

cf = plt.contourf(lons, lats, data_t2m, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.03)
cb.set_label('T2m (°C)', fontsize=10, rotation=90)

### Plot a complex variable like wind direction

Another exemple with more complex variable like wind direction with Cartopy library.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

ax = plt.axes(projection=ccrs.PlateCarree())
bounds = [np.min(lons), np.max(lons), np.min(lats), np.max(lats)]
ax.set_extent(bounds, crs=ccrs.PlateCarree())
ax.coastlines()

ax.stock_img()

lon_formatter = LongitudeFormatter(zero_direction_label=True, number_format='.0f')
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)

# Select every 10 values to keep the map readable
ax.barbs(lons[::10, ::10],lats[::10, ::10],
          data_uwind[::10, ::10],data_vwind[::10, ::10],
          transform=ccrs.PlateCarree())
ax.set_title("Wind Direction");

### Plot variable on S2 satellite image like snow mask

Here, an exemple of plot with an array like snow mask on another array like a S2 satellite image.

In [ ]:
from osgeo import gdal, osr
import pyproj 

"""
# We previously took a S2 image and resampled it to be lighter to work with
image = "data/S2B_MSIL1C_20240112T104319_N0510_R008_T31TDH_20240112T214056.SAFE/GRANULE/L1C_T31TDH_A035784_20240112T104318/IMG_DATA/T31TDH_20240112T104319_B04.jp2"
ds = gdal.Open(image)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
options = gdal.WarpOptions(xRes= gt[1]*20, yRes= gt[5] * 20)
image_res = 'data/tmp/T31TDH_20240112T104319_B04_resampled.jp2'
gdal.Warp(image_res, image, options=options)
"""

image_res = 'data/T31TDH_20240112T104319_B04_resampled.jp2'

In [ ]:
import rasterio

dataset = rasterio.open(image_res)

# Get image information
epsg = dataset.crs
img_extent = [dataset.bounds[0], dataset.bounds[2], dataset.bounds[1], dataset.bounds[3]] 

proj = pyproj.Transformer.from_crs(epsg, 4326, always_xy=True)
min_lon, max_lat = proj.transform(img_extent[0], img_extent[3])
max_lon, min_lat = proj.transform(img_extent[1], img_extent[2])
img_extent = (min_lon, max_lon, min_lat, max_lat) 

# Add a threshold to the image to be visually comprehensive
img = plt.imread(image_res)
img = np.array(img)
img[img > 10000] = 10000 # change plot dynamique

# Variable preprocessing 
pixel_size = 0.1
var_extent = (np.min(lons), np.max(lons) - pixel_size, np.min(lats), np.max(lats) + pixel_size)

In [ ]:
# Create a the snow mask
# snow = 1 ; no_snow = nan
data_snowc_mask = data_snowc.copy()
data_snowc_mask[data_snowc_mask > 0.8] = 1
data_snowc_mask[data_snowc_mask <= 0.8] = np.nan
data_snowc_mask = data_snowc_mask.filled(np.nan)

#### Using Cartopy


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Make the map
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title("Snow Mask")
ax.set_extent(img_extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('110m'), linewidth=0.75)

# Add S2 image
ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(), cmap="gray", alpha=1.0)

# Add snow mask (nan values are transparent)
ax.imshow(data_snowc_mask, origin='upper', extent=var_extent, transform=ccrs.PlateCarree(), alpha=0.6)

# Add coordinates legends
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

plt.show()

#### Using Folium

In [ ]:
import folium

m = folium.Map([42.3576304959657, 1.7656944498250533], zoom_start=8)

# Add S2 image
folium.raster_layers.ImageOverlay(
    image=img,
    name="S2 Image",
    bounds=[[img_extent[2],img_extent[0]],[img_extent[3],img_extent[1]]],
    opacity=1,
    zindex=1,
).add_to(m)

# Create a custom colormap with nan values transparent
cmap = plt.colormaps["viridis"]
cmap.set_bad(alpha=0)  

# Add snow mask
folium.raster_layers.ImageOverlay(
    image=data_snowc_mask,
    name="Snow Mask",
    bounds=[[np.min(lats), np.min(lons)],[ np.max(lats) + pixel_size, np.max(lons) - pixel_size]],  
    colormap=cmap,
    opacity=0.6,
    zindex=2,
).add_to(m)

folium.LayerControl().add_to(m)

m

## Analysis example



In [ ]:
# Load data in a pandas 
data = np.array([np.array(lats[:]).flatten(), 
                 np.array(lons[:]).flatten(), 
                 np.array(data_t2m).flatten(),
                 np.array(data_press[:]).flatten(),
                 np.array(data_uwind[:]).flatten(),
                 np.array(data_vwind[:]).flatten(),
                 np.array(data_snowa[:]).flatten(),
                 np.array(data_snowc[:]).flatten(),
                ]).T

df = pd.DataFrame(data=data, columns=["lat", "lon", "t2m", "press", "uwind", "vwind", "snowa", "snowc"])

# And transform it in the geopandas (with a new geometry column containing Point(lon, lat) info)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)

# Display the first 5 values
gdf.head()  # NaN values are present on sea area

### SeaBorn

Seaborn is a Python statistical data visualization library based on matplotlib. 

In [ ]:
import seaborn as sns

# Study of the relationship between temperature (t2m) and the presence of snow (snowc)
# and their histogram
sns.jointplot(data=gdf, x="t2m", y="snowc");

In [ ]:
# Study of the relationship between temperature (t2m) and the latitude (lat)
sns.kdeplot(data=gdf, x="t2m", y="lat");

In [ ]:
# Study of the relationship several variables and their histogram
sns.pairplot(data=gdf[["snowc","snowa", "t2m", "lat"]]);